# Process Fedscope data

In [3]:
import pickle 

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import os
import statsmodels.formula.api as sm
plt.rcParams.update({'font.size': 16})


Load the main data frame 

In [6]:
dfMain = pd.read_csv("../data/Sep18/FACTDATA_SEP2018.TXT",
                  header=0,
                  low_memory=False)
# dfMain = pd.read_csv("/Users/vcy/Dropbox/1 Work/Data bank/Fedscope/2018 Sept/FACTDATA_SEP2018.TXT",
#                   header=0,
#                   low_memory=False)

In [7]:
dfMain

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AA00,11,E,13,NaN,F,0340,2,ES,ES-**,Q,XXXX,2,50,F,1,201809,1,174500.0,15.3
1,AA00,11,H,15,NaN,H,0905,1,ES,ES-**,Q,XXXX,2,50,F,1,201809,1,174500.0,25.7
2,AA00,11,E,15,15,D,0905,1,99,GS-15,P,XXXX,2,30,F,1,201809,1,161746.0,9.0
3,AA00,11,E,15,13,D,0905,1,99,GS-13,J,XXXX,8,30,F,1,201809,1,100203.0,9.9
4,AA00,11,G,15,NaN,D,0301,2,ES,ES-**,Q,XXXX,2,50,F,1,201809,1,174500.0,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100797,ZU00,11,E,17,NaN,D,0301,2,99,AD-00,J,XXXX,8,38,F,1,201809,1,100576.0,5.3
2100798,ZU00,11,J,13,NaN,D,0301,2,99,AD-00,K,XXXX,2,38,F,1,201809,1,116365.0,5.5
2100799,ZU00,11,H,04,NaN,B,0510,1,99,AD-00,F,XXXX,8,38,F,1,201809,1,62796.0,2.5
2100800,ZU00,11,B,13,NaN,A,0301,2,99,AD-00,E,XXXX,8,38,F,1,201809,1,50000.0,0.0


Agency data frame

In [4]:
agy = pd.read_csv("/Users/vcy/Dropbox/1 Work/Data bank/Fedscope/2018 Sept/DTagy.txt", 
                  header=0, 
                  low_memory=False)

Occupations

In [5]:
occ = pd.read_csv("/Users/vcy/Dropbox/1 Work/Data bank/Fedscope/2018 Sept/DTocc.txt",header=0, low_memory=False)

In [6]:
occ.head()

,OCCTYP,OCCTYPT,OCCFAM,OCCFAMT,OCC,OCCT
0,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0006,0006-CORRECTIONAL INSTITUTION ADMINISTRATION
1,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0007,0007-CORRECTIONAL OFFICER
2,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0017,0017-EXPLOSIVES SAFETY
3,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0018,0018-SAFETY AND OCCUPATIONAL HEALTH MANAGEMENT
4,1,White Collar,00,00xx-MISCELLANEOUS OCCUPATIONS,0019,0019-SAFETY TECHNICIAN


## Restructure data

In [7]:
merged = pd.merge(dfMain, agy[["AGYSUB", "AGY"]], on = "AGYSUB", how = "left")

In [8]:
total = merged.groupby("AGY")["EMPLOYMENT"].count()

In [9]:
occBreakdown = merged.groupby(["AGY", "OCC"])["EMPLOYMENT"].count().reset_index()

In [10]:
occBreakdown.head()

,AGY,OCC,EMPLOYMENT
0,AA,0301,1
1,AA,0340,1
2,AA,0905,8
3,AA,2210,1
4,AB,0080,1


Save to pickle

In [16]:
occBreakdown.to_pickle('fedOccFine2018')

## Get total

In [12]:
total = total.reset_index().rename(columns = {"EMPLOYMENT": "total"})

In [13]:
total.head()

,AGY,total
0,AA,11
1,AB,79
2,AF,169381
3,AG,90382
4,AH,390


## Get bureaucrat counts

In [14]:
# a dictionary for various bureaucrats
Bdict = {"secretaries": ["0318"], 
         "HR": ["0201", "0203", "0299"], 
         "legal": list(occ[occ.OCCFAM == "09"].OCC.unique()),  # these are defined by family
         "admins" : list(occ[occ.OCCFAM == "03"].OCC.unique()),
         "accounting" : list(occ[occ.OCCFAM == "05"].OCC.unique()),
         "maintenance" : list(occ[occ.OCCFAM == "47"].OCC.unique()), # GENERAL MAINTENANCE AND OPERATIONS 
        "facilities_service" : list(occ[occ.OCCFAM == "16"].OCC.unique()), # EQUIPMENT, FACILITIES, AND SERVICES GROUP
         "custodian": ["3566"]
        }

In [15]:
# Function to get varioys bureaucracts in agency
def getB(occBreakdown, key, Bdict):
    temp = occBreakdown[occBreakdown.OCC.isin(Bdict[key])]
    B = temp.groupby("AGY").EMPLOYMENT.sum().reset_index()
    B.rename(columns = {"EMPLOYMENT": key}, inplace = True)
    B = B.set_index("AGY")
    return B

In [16]:
Blist = []
for i, key in enumerate(Bdict.keys()):
    B = getB(occBreakdown, key, Bdict)
    Blist.append(B)

In [17]:
agyB = pd.concat(Blist, axis = 1, sort=False).fillna(0).reset_index().rename(columns = {"index": "AGY"})

In [18]:
agyB.head()

,AGY,secretaries,HR,legal,admins,accounting,maintenance,facilities_service,custodian
0,AF,1975.0,4371.0,1096.0,30406,7644.0,1237.0,2990.0,4.0
1,AG,400.0,1505.0,386.0,11550,2839.0,334.0,94.0,44.0
2,AH,1.0,14.0,13.0,196,24.0,0.0,0.0,0.0
3,AM,11.0,68.0,110.0,1643,355.0,0.0,1.0,0.0
4,AR,1506.0,9750.0,2590.0,48657,9316.0,2284.0,3505.0,7.0


In [19]:
# merge total into the bureaucrat data frame
B = pd.merge(total, agyB, on = "AGY")

In [20]:
B.head()

,AGY,total,secretaries,HR,legal,admins,accounting,maintenance,facilities_service,custodian
0,AA,11,0.0,0.0,8.0,2,0.0,0.0,0.0,0.0
1,AB,79,0.0,2.0,1.0,17,7.0,0.0,35.0,0.0
2,AF,169381,1975.0,4371.0,1096.0,30406,7644.0,1237.0,2990.0,4.0
3,AG,90382,400.0,1505.0,386.0,11550,2839.0,334.0,94.0,44.0
4,AH,390,1.0,14.0,13.0,196,24.0,0.0,0.0,0.0


## Get supervisory status; supertyp; 1, 2 vs 3..7

In [21]:
superv = pd.read_csv("/Users/vcy/Dropbox/1 Work/Data bank/Fedscope/2018 Sept/DTsuper.txt",header=0, low_memory=False)
superv.head()

,SUPERTYP,SUPERTYPT,SUPERVIS,SUPERVIST
0,1,Supervisor,2,2-SUPERVISOR OR MANAGER
1,2,Leader,6,6-LEADER
2,2,Leader,7,7-TEAM LEADER
3,3,Non-Supervisor,4,4-SUPERVISOR (CSRA)
4,3,Non-Supervisor,5,5-MANAGEMENT OFFICIAL (CSRA)
